In [98]:
import torch.utils.data as data
import numpy as np
#from utils import process_feat
import torch
import numpy as np
import os
import glob
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import torch.nn as nn
import torch.nn.init as torch_init

import torch.nn.functional as F
import random
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from torch.nn.functional import cosine_similarity
from sklearn.metrics import confusion_matrix, roc_auc_score
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import time
from itertools import cycle
from sklearn.metrics import roc_curve, auc
torch.set_default_tensor_type('torch.cuda.FloatTensor')
import matplotlib.pyplot as plt
from random import sample
#from kmeans_pytorch import kmeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import logging
import sys

In [120]:
import warnings
warnings.simplefilter("ignore")


In [121]:

class ucfdataset(Dataset):
    def __init__(self, data_dir, test=False, annotation_file=None):
        self.data_dir = data_dir
        self.class_folders = sorted(os.listdir(data_dir))
        self.test = test
        self.annotation_file = annotation_file

        self.file_paths = []
        for class_folder in self.class_folders:
            class_dir = os.path.join(data_dir, class_folder)
            file_paths = [os.path.join(class_dir, file_name) for file_name in os.listdir(class_dir) if file_name.endswith("_x264.npy") and not file_name.startswith(".")]
            self.file_paths.extend(file_paths)


        if self.test and self.annotation_file:
            self.annotations = self.load_annotations(self.annotation_file)



    def load_annotations(self, annotation_file):
        annotations = {}
        with open(annotation_file, 'r') as f:
            for line in f:
                video_name, event, start_frame1, end_frame1, start_frame2, end_frame2 = line.strip().split()
                start_frame1 = int(start_frame1) if start_frame1 != "-1" else -1
                end_frame1 = int(end_frame1) if end_frame1 != "-1" else -1
                start_frame2 = int(start_frame2) if start_frame2 != "-1" else -1
                end_frame2 = int(end_frame2) if end_frame2 != "-1" else -1
                annotations[video_name] = (event, start_frame1, end_frame1, start_frame2, end_frame2)
            return annotations

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):

        file_path = self.file_paths[index]
        video_crops = np.load(file_path)
        labels = np.zeros(len(video_crops))
#         vid=[]

        if self.test:
            video_name = os.path.splitext(os.path.basename(file_path))[0]
            video_name = video_name + ".mp4"


            if video_name in self.annotations:
                event, start_frame1, end_frame1, start_frame2, end_frame2 = self.annotations[video_name]
                init_labels = np.zeros((len(video_crops)*16))
                if start_frame1 != -1:
                    init_labels[start_frame1-1: end_frame1] = 1
                if start_frame2 != -1:
                    init_labels[start_frame2-1: end_frame2] = 1
                segment_size = 16
                for i in range (len(video_crops)):
                    segment= init_labels[i*segment_size : (i+1)*segment_size]
                    num_ones = np.sum(segment)
                    if num_ones >= 7:
                        labels[i] = 1


            seg = torch.tensor(video_crops)
            lbl = torch.tensor(labels)
#             print(len(seg))
#             print(len(lbl))


        else:
            video_name = os.path.splitext(os.path.basename(file_path))[0]


            if video_name.startswith("Normal"):

                labels= np.zeros(len(video_crops))

            else:
                labels=np.ones(len(video_crops))

            batch_size = 256
            num_segments = len(video_crops)
            seg = []
            lbl = []
            selected_label = []
            selected_segment = []
            

            for start_idx in range(0, num_segments, batch_size):
                end_idx = start_idx + batch_size
                selected_segment = video_crops[start_idx:end_idx]
                selected_label = labels[start_idx:end_idx]

                if len(selected_segment)< batch_size:
                    selected_segment = [item for item, idx in zip(cycle(selected_segment), range(batch_size))]
                    selected_label = [item for item, idx in zip(cycle(selected_label), range(batch_size))]


                selected_segments = torch.tensor(selected_segment)
                selected_labels = torch.tensor(selected_label)
                seg.append(selected_segments)
                lbl.append(selected_labels)

#         vid=torch.tensor(video_crops)

        return seg , lbl #, vid


In [122]:
data_dir = '/home/iml1/Desktop/UMAIR/ucf/i3d_features/training'
data_dir2 = '/home/iml1/Desktop/UMAIR/ucf/i3d_features/test'
annotation_file = '/home/iml1/Desktop/UMAIR/ucf/i3d_features/Temporal_Anomaly_Annotation_for_Testing_Videos.txt'
batch_size = 1

train_dataset= ucfdataset(data_dir)
train_dataloader = DataLoader(train_dataset , batch_size = batch_size, shuffle=True, generator=torch.Generator(device='cuda'))

test_dataset = ucfdataset(data_dir2, test=True, annotation_file=annotation_file)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, generator=torch.Generator(device='cuda'))


In [123]:
import numpy as np

# Load a sample .npy file (replace with your actual path)
sample_feature = np.load("/home/iml1/Desktop/UMAIR/ucf/i3d_features/training/Abuse/Abuse027_x264.npy")  

print("Type:", type(sample_feature))  # Should be <class 'numpy.ndarray'>
print("Shape:", sample_feature.shape)  # e.g., (1024,) or (16, 1024) for sequential features

Type: <class 'numpy.ndarray'>
Shape: (273, 1024)


In [125]:
class NSB1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NSB1, self).__init__()
        self.fc = nn.Linear(input_size, hidden_size)

    def forward(self, x):
        x = self.fc(x)
        x = F.softmax(x, dim=0)
        return x

In [126]:
class NSB2(nn.Module):
    def __init__(self, input_size2, output_size2):
        super(NSB2, self).__init__()
        self.fc = nn.Linear(input_size2, output_size2)

    def forward(self, x2):
        x2 = self.fc(x2)
        x2 = F.softmax(x2 , dim=0)
        return x2

In [127]:
class BBN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.6):
        super(BBN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(output_size, 1)
        self.nsb1=NSB1(input_size,hidden_size)
        self.nsb2=NSB2(hidden_size,output_size)

    def forward(self, x):
        x1 = self.fc1(x)
        nsb1 = self.nsb1(x)
        
        fc1_res = torch.mul(x1, nsb1)
        x3 = F.relu(fc1_res)
        x3 = self.dropout1(x3)
        
        x4 = self.fc2(x3)
        nsb2 = self.nsb2(x3)
        
        
        fc2_res = torch.mul(x4,nsb2)
        x5 =  F.relu(fc2_res)
        x5 = self.dropout2(x5)
        x6 = self.fc3(x5)
        x6 = torch.sigmoid(x6)
        
        
#         nsb1_out= nsb1.detach().cpu().numpy()
#         print("nsb1 output size",nsb1.size(),"maximum value",np.max(nsb1_out), "minimum value", np.min(nsb1_out))
#         plt.matshow(nsb1_out)
#         plt.colorbar()
#         plt.show()
        
#         nsb2_out= nsb2.detach().cpu().numpy()
#         print("nsb2 output size",nsb2.size(),"maximum value",np.max(nsb2_out), "minimum value", np.min(nsb2_out))
#         plt.matshow(nsb2_out)
#         plt.colorbar()
#         plt.show()
        
        
        return x6


In [128]:
class Clustering(nn.Module):
    def __init__(self, n_clusters=2):
        super(Clustering, self).__init__()
        self.n_clusters = n_clusters
        self.alpha = 0.7
        self.beta = 0.25
        

    def clusters(self, c_vid_seg, labels):
        
        scaler = StandardScaler()
        vid_seg=c_vid_seg.clone()
    
        normalize_data = scaler.fit_transform(vid_seg.detach().cpu().numpy())
    
        
        kmeans = KMeans(n_clusters=self.n_clusters)
        kmeans.fit(normalize_data)

        cluster_centers = kmeans.cluster_centers_
        
        cluster_centers_tensor = torch.tensor(cluster_centers)
        
        normalized_data = torch.tensor(normalize_data)
        
        if labels == 0:
            cluster_loss = self.normal_loss(cluster_centers_tensor, normalized_data)
        else:
            cosine_sim_cc = F.cosine_similarity(cluster_centers_tensor[0].unsqueeze(0),cluster_centers_tensor[1].unsqueeze(0))
            selected_features_cluster1 = []
            selected_features_cluster2 = []
            c1=0
            c2=0
            for i, point in enumerate(normalized_data):
                cluster_label = kmeans.labels_[i]
                point_tensor = torch.tensor(point)
                
                 
            
                sim_to_cluster_center = 1 - F.cosine_similarity(point_tensor, cluster_centers_tensor[cluster_label], dim=0)
                
                if cluster_label==0 and c1==0:
                    temp = point_tensor
                    c1+=1
                if cluster_label == 1 and c2==0:
                    temp1 = point_tensor
                    c2+=1
                
                if cluster_label == 0 and sim_to_cluster_center < self.beta * (1 - cosine_sim_cc):
                    selected_features_cluster1.append(point_tensor)
                elif cluster_label == 1 and sim_to_cluster_center < self.beta * (1 - cosine_sim_cc):
                    selected_features_cluster2.append(point_tensor)
                    
            if len(selected_features_cluster1)==0:
                selected_features_cluster1=temp
            if len(selected_features_cluster2)==0:
                selected_features_cluster2=temp1
                
                
                
#             if len(selected_features_cluster1)>1:
#                 selected_features_cluster1 = torch.stack(selected_features_cluster1)
#             if len(selected_features_cluster2)>1:
#                 selected_features_cluster2 = torch.stack(selected_features_cluster2)

            cluster_loss = self.anomaly_loss(cluster_centers_tensor, selected_features_cluster1,selected_features_cluster2)
            

        return cluster_loss

    def normal_loss(self, cluster_centers, normalized_data):
        all_lcn = []
        mean_center = torch.mean(cluster_centers, dim=0).to(device)

        for feature in normalized_data:
            temp = 1 - F.cosine_similarity(feature.reshape(1,-1), mean_center.reshape(1,-1))
            all_lcn.append(temp)
        l_c_n = (torch.mean(torch.stack(all_lcn).squeeze(1)))
    
        return l_c_n
    
        
    def anomaly_loss(self, cluster_centers, selected_features_cluster1,selected_features_cluster2):
        lcc1=[]
        lcc2=[]
        lcd1=[]
        lcd2=[]
        
        for features in selected_features_cluster1:
            similarity = 1 - torch.cosine_similarity(features.unsqueeze(0), cluster_centers[0].unsqueeze(0))
            lcc1.append(similarity)
            similarity2 = 1 + torch.cosine_similarity(features.unsqueeze(0), cluster_centers[1].unsqueeze(0))
            lcd1.append(similarity2)
                       
        for feature in selected_features_cluster2:
            similarity3 = 1 - torch.cosine_similarity(feature.unsqueeze(0), cluster_centers[1].unsqueeze(0))
            lcc2.append(similarity3)
            similarity4 = 1 + torch.cosine_similarity(feature.unsqueeze(0), cluster_centers[0].unsqueeze(0))
            lcd2.append(similarity4)


                
        lcc = torch.mean(torch.stack(lcc1 + lcc2))
        lcd = torch.mean(torch.stack(lcd1 + lcd2))
        l_c_a = self.alpha * lcc + (1 - self.alpha) * lcd
        return l_c_a


In [129]:
class LossCalculator:
    def __init__(self, pred_y, y,clustering_loss):
        self.lembda1 = 0.00008
        self.lembda2 =0.00008
        self.y=y
        pred_y = pred_y.reshape(y.shape)
        self.clustering_loss=clustering_loss

    def regression_loss(self, pred_y , y):
        r_loss = nn.MSELoss()
        l_reg = r_loss(pred_y.to(torch.float32),y.to(torch.float32))
        return l_reg

    def temporal_smoothness_loss(self, pred_y):
        length = pred_y.size(0)
        y_i = pred_y[0:length-1]

        y_i_plus_1 = pred_y[1:length]


        squared_diff = torch.pow(y_i - y_i_plus_1, 2)
        sum_squared_diff = torch.sum(squared_diff)
        temp_smooth_loss = (sum_squared_diff)/(length-1)

        return temp_smooth_loss

    def sparsity_loss(self, pred_y):
        return torch.mean(pred_y)



    def total_loss(self, pred_y, y,):
        l_reg = self.regression_loss(pred_y,y)
        temp_smooth_loss = self.temporal_smoothness_loss(pred_y)
        spars_loss = self.sparsity_loss(pred_y)
        total_loss = l_reg + self.lembda1*(spars_loss + temp_smooth_loss) + self.lembda2*(self.clustering_loss)

        return total_loss

In [130]:
input_size =1024
hidden_size = 512
#hidden_size2=256
output_size = 32


model = BBN(input_size, hidden_size ,  output_size, dropout_rate=0.6)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model.to(device)

print(model)

BBN(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (dropout1): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=512, out_features=32, bias=True)
  (dropout2): Dropout(p=0.6, inplace=False)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (nsb1): NSB1(
    (fc): Linear(in_features=1024, out_features=512, bias=True)
  )
  (nsb2): NSB2(
    (fc): Linear(in_features=512, out_features=32, bias=True)
  )
)


In [71]:
#model.load_state_dict(torch.load('/media/itucvl/Local Disk/Ramsha/bbn/models/bbn_best.pth'))

In [72]:
# weights= torch.load('/media/itucvl/Local Disk/Ramsha/bbn/models/bbn_best.pth')                                                         
# model.fc1_weight=weights['fc1.weight']
# model.fc1_bias=weights['fc1.bias']
# model.fc2_weight=weights['fc2.weight']
# model.fc2_bias=weights['fc2.bias']
# model.fc3_weight=weights['fc3.weight']
# model.fc3_bias=weights['fc3.bias']

In [73]:
!pip install prettytable

In [131]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)


+----------------+------------+
|    Modules     | Parameters |
+----------------+------------+
|   fc1.weight   |   524288   |
|    fc1.bias    |    512     |
|   fc2.weight   |   16384    |
|    fc2.bias    |     32     |
|   fc3.weight   |     32     |
|    fc3.bias    |     1      |
| nsb1.fc.weight |   524288   |
|  nsb1.fc.bias  |    512     |
| nsb2.fc.weight |   16384    |
|  nsb2.fc.bias  |     32     |
+----------------+------------+
Total Trainable Params: 1082465


1082465

In [132]:
model.to(device)

BBN(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (dropout1): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=512, out_features=32, bias=True)
  (dropout2): Dropout(p=0.6, inplace=False)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
  (nsb1): NSB1(
    (fc): Linear(in_features=1024, out_features=512, bias=True)
  )
  (nsb2): NSB2(
    (fc): Linear(in_features=512, out_features=32, bias=True)
  )
)

In [133]:
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [134]:
def auc_calculator(true_labels, predicted_scores):
    true_labels_flatten = [item.cpu().numpy() for sublist in true_labels for item in sublist]
    predicted_scores_flatten = [item.cpu().numpy() for sublist in predicted_scores for item in sublist]

    true = np.array(true_labels_flatten)
    pre = np.array(predicted_scores_flatten)
    # print("true_labels_flatten and predicted_scores_flatten shapes", true.shape , pre.shape )
    fpr, tpr, threshold = roc_curve(true_labels_flatten, predicted_scores_flatten)

    auc_res = auc(fpr, tpr)
    
#     plt.plot(fpr,tpr , label='AUC Curve')
#     plt.title('AUC Curve')
#     plt.xlabel('fpr')
#     plt.ylabel('tpr')
#     plt.grid(True)
#     plt.show()
    return auc_res

In [78]:
# se = train_dataset[1][0]
# le = train_dataset[1][1]
# torch_labels=torch.stack(le)
# torch_labels=torch_labels.squeeze(1)
# train_vid_label=torch.sum(torch_labels)
# if train_vid_label>1:
#     clus_lbl=1
# else:
#     clus_lbl=0

# torch_segments=torch.stack(se)
# torch_segments=torch_segments.squeeze(1).view(-1,1024).to(device)
# fc1_seg = model.fc1(torch_segments)
# print("fc1_seg shape", fc1_seg.shape)

# clustering=Clustering()
# cluster_loss=clustering.clusters(fc1_seg,clus_lbl)
# print(cluster_loss)

In [ ]:
from tqdm import tqdm

lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80000], gamma=0.1)
th = 0.5
best_res = 0.0
count = 0
min_AUC = 0
losses_curve = []
path = '/home/iml1/Desktop/UMAIR/ucf/ModelsJupyter'
# Training loop
epoches = 15
all_auc =[]
tp_loss=[]
_loss=[]
cou=0

for epoch in range(epoches):
    start_time = time.time()
    model.train()
    all_losses =[]
    epoch_loss =[]
    
    # Wrap train_dataloader with tqdm for progress bar
    train_iter = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epoches}', leave=False)
    for segments, labels in train_iter:
        torch_labels=torch.stack(labels)
        torch_labels=torch_labels.squeeze(1)
        train_vid_label=torch.sum(torch_labels)
        if train_vid_label>1:
            clus_lbl=1
        else:
            clus_lbl=0
        
        torch_segments=torch.stack(segments)
        torch_segments=torch_segments.squeeze(1).view(-1,1024).to(device)
    
        indices = random.sample(range(len(segments)), len(segments))
        for index in indices:
            seg_output = model.fc1(torch_segments)
            clustering=Clustering()
            cluster_loss=clustering.clusters(seg_output,clus_lbl)
            train_segments = segments[index].to(device)
            train_labels = labels[index].to(device)
            pred_y = model(train_segments.squeeze(0))
            pred_y = pred_y.squeeze(0)
            train_labels = train_labels.T
            loss_calculator = LossCalculator(pred_y, train_labels,cluster_loss)
            losses = loss_calculator.total_loss(pred_y, train_labels)
            _loss.append(losses.detach().cpu().numpy())
            all_losses.append(losses.detach().cpu().numpy())
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            lr_scheduler.step()
            
        # Update progress bar with current loss
        train_iter.set_postfix({'Loss': np.mean(all_losses[-len(indices):])})
            
    mean_loss = np.mean(all_losses)
    losses_curve.append(mean_loss)
    torch.cuda.empty_cache()        
    model.eval()
    true_labels = []
    predicted_scores = []
    t_video=[]
    
    # Evaluation progress bar
    with torch.no_grad():
        test_iter = tqdm(test_dataset, desc='Evaluating', leave=False)
        test_loss=[]
        for segments, labels in test_iter:
            segment = segments.to(device)
            te_label = labels.to(device)
            te_pred_y = model(segment)
            
            t_label=te_label.reshape(len(te_label),1)
            t_pred_y=te_pred_y.reshape(len(te_pred_y),1)
            true_labels.append(t_label)
            predicted_scores.append(t_pred_y)

            del t_label , t_pred_y
            torch.cuda.empty_cache()

    auc_res = auc_calculator(true_labels, predicted_scores)

    del true_labels , predicted_scores
    torch.cuda.empty_cache()

    if auc_res > min_AUC:
        min_AUC = auc_res
        model_path =  os.path.join(path,f'best_clustering.pth')
        torch.save(model.state_dict() , model_path)

    all_auc.append(auc_res)

    if auc_res > best_res:
        best_res = auc_res

    end_time = time.time()
    epoch_time = end_time - start_time
    model_path = os.path.join('/home/iml1/Desktop/UMAIR/ucf/ModelsJupyter', f'{epoch+1}.pth')
    torch.save(model.state_dict(), model_path)
    
    # Print epoch summary
    print(f"\nEpoch [{epoch+1}/{epoches}] - Test AUC: {auc_res:.4f} - Training Loss: {mean_loss:.4f} - Time taken: {epoch_time:.2f} seconds")


Epoch [1/15] - Test AUC: 0.5550 - Training Loss: 0.1897 - Time taken: 8846.97 seconds



Epoch [2/15] - Test AUC: 0.5435 - Training Loss: 0.1599 - Time taken: 8242.49 seconds



Epoch [3/15] - Test AUC: 0.5542 - Training Loss: 0.1381 - Time taken: 8266.64 seconds



Epoch [4/15] - Test AUC: 0.5950 - Training Loss: 0.1196 - Time taken: 8382.35 seconds



Epoch [5/15] - Test AUC: 0.5984 - Training Loss: 0.1010 - Time taken: 8240.87 seconds


...


Epoch [7/15] - Test AUC: 0.6405 - Training Loss: 0.0797 - Time taken: 8208.00 seconds



Epoch [8/15] - Test AUC: 0.6838 - Training Loss: 0.0736 - Time taken: 8172.55 seconds



Epoch [9/15] - Test AUC: 0.6880 - Training Loss: 0.0677 - Time taken: 8630.68 seconds



Epoch [10/15] - Test AUC: 0.6881 - Training Loss: 0.0627 - Time taken: 8686.95 seconds


Epoch 11/15:  50%|█████     | 810/1610 [46:07<08:07,  1.64it/s, Loss=0.0215]     

In [ ]:
model.load_state_dict(torch.load("/media/itucvl/Local Disk/Ramsha/bbn/models/random_nsb2.pth"))

In [ ]:
model.eval()
S = test_dataset[100][0].to(device)
L = test_dataset[100][1].to(device)
print(L)
P=model(S)

In [ ]:
model.load_state_dict(torch.load("/media/itucvl/Local Disk/Ramsha/random_nsb2/1.pth"))

In [ ]:
model.eval()
S = test_dataset[100][0].to(device)
L = test_dataset[100][1].to(device)
print(L)
P=model(S)

In [ ]:
ls = [item for item in losses_curve]

In [ ]:
lsp = np.array(losses_curve)
lsp.shape

In [ ]:
ls2 = [items for items in tp_loss]
lsp2 = np.array(ls2)
# lsp2.shape

In [ ]:
itera = np.array(range(1 , len(ls)+1))

In [ ]:
# itera.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(itera,ls , label='Training loss')
plt.plot(itera,tp_loss, label='validation Loss')
plt.title('Loss Curve')
plt.xlabel('itera')
plt.ylabel('loss')
plt.grid(True)
plt.show()

In [ ]:
# import csv
# csv_filename='trainig_loss.csv'
# with open (csv_filename, mode='w',newline='') as csv_file:
#     writer=csv.writer(csv_file)
#     writer.writerow(['training_loss'])
    
#     for val in _loss:
#         writer.writerow([val])
    
    

In [ ]:
model.eval()

In [ ]:
true_labels = []
predicted_scores=[]
with torch.no_grad():
    for segments, labels in test_dataloader:
        for i in range (len(segments)):
            segment = segments[i].to(device)
            t_label = labels[i].to(device)
            t_pred_y = model(segment)

            del segment


            true_labels.append(t_label)
            predicted_scores.append(t_pred_y)


            del t_label , t_pred_y
            torch.cuda.empty_cache()

auc_res = auc_calculator(true_labels, predicted_scores)

In [ ]:
print(np.max(all_auc))

In [ ]:
epo = list(range(1 , len(all_auc)+1))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(epo,all_auc)

plt.title('Learning Curve')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.grid(True)
plt.show()

In [ ]:
# arr = [1,2,3,4,5,6,77,8]
# leng = len(arr)
# a = arr[1:leng]
# b = arr[:leng-1]
# print(a)
# print(b)
# for h in range (len(arr)-1):
#     print(arr[h] , arr[h+1] , "=" , arr[h]-arr[h+1])


In [ ]:
model_path = os.path.join('/content/drive/MyDrive/i3d_features/i3d_features', f'{150}.pth')

In [ ]:
torch.save(model.state_dict(), model_path)

In [ ]:
import pickle

In [ ]:
data_to_save = {
    'all_auc': all_auc,
    'trainingloss': ls,
    'validationloss':lsp
}

In [ ]:
with open('/content/drive/MyDrive/i3d_features/variables.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

In [ ]:
# from random import sample
# list_=[1,2,3,4,5]
# ind = sample(list_, 5)
# ind